In [ ]:
pip install peewee;

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import requests
import pandas as pd
import peewee
from peewee import *
import numpy as np
import xarray as xr
import os

In [ ]:
def prepararDF(df, codigo):
  # Muda o tipo da coluna 'Data' para datetime (string --> datetime)
  df['Data'] = df['DataHora'][:10]
  df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # Exclui colunas desnecessárias do dataframe
  listColumnsDrop = []
  for i in range(77):
    if i > 44:
      listColumnsDrop += [i]
  listColumnsDrop += [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
  df = df.drop(df.columns[listColumnsDrop], axis=1)
  
  # Avalia se existe uma mesma data com diferentes níveis de consistência
  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # Reseta os índices para passar "Data" como coluna e utilizar a mesma para apagar as linhas duplicadas
  df = df.reset_index()

  # apagas as linhas duplicadas, dando preferência ao dado consistido ('NivelConsistencia' = 2)
  df = df.drop_duplicates(subset='Data', keep='last', inplace=False)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # transforma as colunas de chuva em linhas com as respectivas datas
  # Este processo transforma todas as colunas em linhas, gerando um problema para os meses que possuem menos de 31 dias
  # Dessa forma, o dia 1 de um mês armazenava a chuva do dia 1 e a chuva do dia 31 (linha duplicada com valor NaN)
  # Correção explicada durante o processo:

  # usa a função .melt() para transformar todas as colunas em linhas. São mantidas as colunas ['Data', 'NivelConsistencia']
  # São criadas duas novas colunas: 'Dia' que recebe o nome da coluna que virou linha e 'Chuva' que recebe o valor que era armazenado na coluna
  df = df.reset_index().melt(id_vars=['Data', 'NivelConsistencia'], var_name='Dia', value_name='Chuva')

  # Fiz uma cópia da minha data em formato de texto
  df['Data1']=df['Data'].astype(str)

  # Apaguei os últimos 2 caracteres, assim ficou somente o texto com %Y-%m- ('1989-11-')
  df['Data1'] = df['Data1'].str[:-2]

  # Fiz uma cópia da coluna 'Dia' (exe: 'Chuva03'), a 'função .str.extract('(\d+)', expand=False)' deixa apenas os números
  # (então a coluna 'Dia1' armazena a string '03') ('Chuva03' --> '03')  
  df['Dia1'] = df['Dia'].str.extract('(\d+)', expand=False)
  
  # praticamente a mesma coisa da linha anterior, mudando apenas que substitui a própria coluna e o formato é int ('Chuva03' --> 3)
  df['Dia'] = df['Dia'].str.extract('(\d+)', expand=False).astype(int)

  # Como foram criadas várias linhas, tem vários índices repetidos da data, esta função soma os dias correspondente
  # aos da chuva guardado na coluna 'Dia' na data do índice (aqui é gerado o erro dos 31 dias)
  df['Data'] = df.apply(lambda x: x['Data'] + pd.DateOffset(days=x['Dia']-1), axis=1)

  # Aqui é feita uma concatenação do tipo '1989-11-'+'03' formando uma data.
  # Mas aqui também forma datas como '1989-02-31'
  df['Data1'] = df['Data1'] + df['Dia1']

  # A maioria das datas das colunas 'Data' e 'Data1' serão iguais, mas datas como '1989-02-31'
  # vão gerar valor NaT pois é um erro (exatamente nas linhas duplicadas)
  df['Data1'] = pd.to_datetime(df['Data1'], format='%Y-%m-%d', errors='coerce')

  # Aqui é apagada todas as linhas em que o erro NaT aparece na coluna 'Data1' (removendo os valores duplicados)
  df.dropna(inplace=True, subset=['Data1'])

  # reorganiza as colunas
  df = df[['Data', 'NivelConsistencia', 'Chuva']]

  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice novamente
  #df.set_index(['Data'], inplace=True)
  df['Codigo'] = codigo
  return df

In [ ]:
# API para solicitar dados de estações sem telemetria
estacoes = ['936003','936004','936014']
frames = []
for estacao in estacoes:
  url = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica'
  params = {
      'codEstacao': estacao,
      'dataInicio': '',
      'dataFim': '',
      'tipoDados': '2',
      'nivelConsistencia': '',
  }
  try:
    response = requests.get(url, params=params)
    xml = response.text
    df1 = pd.read_xml(xml,
                      xpath="//SerieHistorica",
                      namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
  except ValueError:
    pass
  else:
    response = requests.get(url, params=params)
    xml = response.text
    df1 = pd.read_xml(xml,
                      xpath="//SerieHistorica",
                      namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
    df1 = prepararDF(df1, estacao)
    df1 = df1.dropna(subset=['Chuva'])
    #df1.reset_index()
    frames += [df1]
print(f'Foram carregadas {len(frames)} estações')

Foram carregadas 4 estações


In [ ]:
df_936003 = frames[0]
df_936004 = frames[1]
df_936014 = frames[2]

Banco de dados

In [ ]:
db = peewee.SqliteDatabase('teste2.db')

class BaseModel(peewee.Model):
    
    class Meta:
        database = db

class Data(BaseModel):

    data = peewee.DateField(unique=True)
db.create_tables([Data])

class Estacao(BaseModel):
    nivel_consistencia = IntegerField()
    chuva = FloatField()
    codigo_estacao = peewee.CharField()

    # Chave estrangeira para a tabela Data
    date = peewee.ForeignKeyField(Data)
db.create_tables([Estacao])

In [ ]:
if __name__ == '__main__':
    try:
        Data.create_table()
        print("Tabela 'Data' criada com sucesso!")
    except peewee.OperationalError:
        print("Tabela 'Data' ja existe!")

    try:
        Estacao.create_table()
        print("Tabela 'Estacao' criada com sucesso!")
    except peewee.OperationalError:
        print("Tabela 'Estacao' ja existe!")

Tabela 'Data' criada com sucesso!
Tabela 'Estacao' criada com sucesso!


In [ ]:
# iterando pelos dataframes
for df in [df_936003, df_936004, df_936014]:
    for index, row in df.iterrows():
        # criando o objeto Data (se ainda não existe)
        data, created = Data.get_or_create(data=row['Data'])
        
        # criando o objeto Estacao com as informações da linha atual do dataframe
        Estacao.create(nivel_consistencia=row['NivelConsistencia'], chuva=row['Chuva'], codigo_estacao=row['Codigo'], date=data)

In [ ]:
dados = Estacao.select().join(Data).where(Data.data=='1989-01-02')

# Exibe a quantidade de registros que corresponde a nossa pesquisa
print(f'{dados.count()} estações')

for dado in dados:
    print(dado.chuva, dado.codigo_estacao, dado.nivel_consistencia)

3 estações
0.0 936003 1
19.0 936004 1
0.0 936014 1
